In [1]:
import gzip
import random
import numpy as np
import ujson
from datetime import datetime
from collections import defaultdict
import StringIO
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield l

In [2]:
# process comments
countP = defaultdict(int)
countC = defaultdict(int)
countI = defaultdict(int)
rec=set()
cc=0
for l in open('RC_2017-03'):
    l = ujson.loads(l)
    if (l['author'],l['link_id']) in rec: continue
    rec.add((l['author'],l['link_id']))
    countC[l['author']]+=1
    countI[l['link_id']]+=1
    if len(l['id'])!=7: print '!'
    cc+=1
    if cc%10000000==0: print cc
print cc

10000000
20000000
30000000
40000000
48290438


In [3]:
# process submissions
validI1=set(countI.keys())
validI2=set()
UB=dict()
a=0
b=0
cc=0
for l in open('RS_2017-03'):
    cc+=1
    l = ujson.loads(l)
    i = 't3_'+l['id']
    ub = l['author']    
    if i in countI:
        countP[l['author']]+=1
        validI2.add('t3_'+l['id'])
        UB[i] = ub
print cc

9616340


In [4]:
len(set(countP.keys()).intersection(set(countC.keys())))

1331580

In [5]:
validI=validI1.intersection(validI2)

In [6]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = defaultdict(dict)
for (ua,i) in rec:
    if not i in validI: continue
    ub = UB[i]
    # filtering
    if countC[ua]<10 or countP[ua]<10 or countC[ub]<10 or countP[ub]<10 or countI[i]<10: continue
    if ua in usermap: usera = usermap[ua]
    else:
        usera = usernum
        usernum += 1
        usermap[ua] = usera
        User[usera]['consume'] = []
        User[usera]['produce'] = []
    if i in itemmap: itemid = itemmap[i]
    else:
        itemid = itemnum
        itemnum += 1
        itemmap[i] = itemid           
    User[usera]['consume'].append(itemid)        
for (ua,i) in rec:
    if not i in validI: continue
    ub = UB[i]
    if not i in itemmap: continue
    if ub in usermap: userb = usermap[ub]
    else:
        userb = usernum
        usernum += 1
        usermap[ub] = userb
        User[userb]['consume'] = []
        User[userb]['produce'] = []                
    if i in itemmap: itemid = itemmap[i]
    else:
        itemid = itemnum
        itemnum += 1
        itemmap[i] = itemid   
    User[userb]['produce'].append(itemid)

In [8]:
print usernum,itemnum
t=0
p=0
c=0
pros=0
for userid in range(usernum):
    t+=len(User[userid]['consume'])
    if len(User[userid]['consume'])>0: c+=1
    if len(User[userid]['produce'])>0: p+=1
    if len(User[userid]['consume'])>0 and len(User[userid]['produce'])>0: pros+=1
print t,p,c,pros


52654 336743
1786032 45939 52445 45730


In [9]:
dataset=[User,usermap,itemmap,usernum,itemnum]
np.save('meta_core_reddit.npy',dataset)